In [1]:
import sys
import os
home=os.getcwd()
sys.path.append(home+'/Functions')
from ChargeDataSet import *
from MS1IDs import *
from NumpyMSPeaksIdentification import *
from PeaksIdent import *
from ShowDF import *
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt

In [2]:
DataSetName='12111.mzML'

In [3]:
DataSet=ChargeDataSet(DataSetName)

In [4]:
MS1IDVec=MS1IDs(DataSet)

In [196]:
FirstSpectrum=True
for Spec_Id in MS1IDVec[:100]:
    #Spec_Id=int(MS1IDVec[int(Nid)])
    RawSignals=np.array(DataSet[int(Spec_Id)].get_peaks()).T
    if FirstSpectrum:
        RawSpectra=RawSignals
        FirstSpectrum=False
    else:
        RawSpectra=np.append(RawSpectra,RawSignals,axis=0)

In [197]:
RawSpectra=RawSpectra[RawSpectra[:, 0].argsort(),:]

## This one is very important!

In [198]:
SpectrumPeaks=PeaksFindDif(RawSpectra,min_mz=200,max_mz=300)
np.shape(SpectrumPeaks)

(1278, 15)

In [199]:
ShowDF(pd.DataFrame(SpectrumPeaks))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,200.005,0.00243842,92,0.000668846,3.34416,-40,0,1,113916,0,0.000813825,200.001,200.01,199.997,200.012
1,200.016,0.00112702,251,0.000184645,0.923151,-40,0,1,370732,0,0.00074452,200.013,200.019,200.012,200.019
2,200.027,0.00119779,383,0.000158443,0.792109,-40,0,1,631310,0,0.000653721,200.024,200.03,200.024,200.031
3,200.037,0.00107606,15,0.000827082,4.13464,-40,0,1,15889.9,1.63481e-08,0.000803221,200.035,200.039,200.034,200.041
4,200.045,0.00192937,42,0.000804163,4.01992,-40,0,1,48700.3,1.67508e-22,0.000801372,200.041,200.05,200.039,200.05
5,200.091,0.00116836,5,0.00240566,12.0228,-40,0,1,7789.07,0.233392,0.000849665,200.09,200.093,200.088,200.095
6,200.128,0.0011675,26,0.000638229,3.1891,-40,0,1,32100.1,8.86093e-23,0.000838681,200.126,200.131,200.124,200.131
7,200.151,0.00119609,11,0.00114295,5.71042,-40,0,1,13071.8,0.00109874,0.00085005,200.149,200.154,200.148,200.155
8,200.164,0.00114326,115,0.000279279,1.39525,-40,0,1,149879,0,0.000702433,200.162,200.166,200.161,200.168
9,200.183,0.00305613,915,0.000260787,1.30274,-40,0,1,3.99332e+06,0,0.000779763,200.17,200.188,200.174,200.192


In [62]:
#I can go in order and start searching for the signals between my peaks while I evaluate them
ms1_id=MS1IDVec[0]
RawSignals=np.array(DataSet[int(ms1_id)].get_peaks()).T 
Peaks=PeaksIdent(RawSignals)
for idv in np.arange(1,100,4):
    ms1_id=MS1IDVec[idv]
    RawSignals2=np.array(DataSet[int(ms1_id)].get_peaks()).T 
    NewSpectrum=RawSignals2.copy()
    DeviationTimes=4
    for peak in Peaks:
        min_mz_peak=peak[0]-DeviationTimes*peak[1]
        max_mz_peak=peak[0]+DeviationTimes*peak[1]
       # print(min_mz_peak,max_mz_peak)
        peakFilter=(NewSpectrum[:,0]>min_mz_peak)&(NewSpectrum[:,0]<max_mz_peak) 
        NewSpectrumFilter=peakFilter
        #NewSpectrum=NewSpectrum[NewSpectrumFilter,:]
        NewSpectrum[NewSpectrumFilter,:]=0
    NewPeaks=PeaksIdent(NewSpectrum)
    Peaks=np.append(Peaks,NewPeaks,axis=0)
    #print(np.shape(Peaks)[0])
    #peakLoc=np.where(peakFilter)[0]
    #print(NewSpectrum[peakLoc,0])